In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
## drop irrelevant columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
##Encode categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [8]:
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_arr = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_cols = onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_encoded_df = pd.DataFrame(geo_arr, columns=geo_cols)
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
##concatenation one hot encoded
data=pd.concat([data.drop("Geography",axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## Divide the dataset into independent and dependent features
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [11]:
# Split the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42,
)

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
## Save the encoders and sscaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Regression Problem Statement

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:
##Build Our ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),## HL1 COnnected with input layer
    Dense(32,activation='relu'),## Hidden layer 2
    Dense(1)## Output layer,Default activation function that is Linear
])

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()



Model: "sequential"

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                   

In [16]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [17]:
##Setup Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [18]:
###------------TRAIN THE MODEL--------------->
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100




250/250 [==============================] - 2s 5ms/step - loss: 100381.2422 - mae: 100381.2422 - val_loss: 98526.8125 - val_mae: 98526.8125
Epoch 2/100
250/250 [==============================] - 2s 5ms/step - loss: 100381.2422 - mae: 100381.2422 - val_loss: 98526.8125 - val_mae: 98526.8125
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99648.4609 - mae: 99648.4609 - val_loss: 97013.4688 - val_mae: 97013.4688
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 99648.4609 - mae: 99648.4609 - val_loss: 97013.4688 - val_mae: 97013.4688
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 96989.7031 - mae: 96989.7031 - val_loss: 93062.0234 - val_mae: 93062.0234
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 96989.7031 - mae: 96989.7031 - val_loss: 93062.0234 - val_mae: 93062.0234
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 91705.8672 - mae: 91705

In [19]:
%load_ext tensorboard

In [22]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 10436), started 0:00:12 ago. (Use '!kill 10436' to kill it.)

In [24]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')


63/63 [==============================] - 0s 2ms/step - loss: 50270.8906 - mae: 50270.8906
Test MAE : 50270.890625
63/63 [==============================] - 0s 2ms/step - loss: 50270.8906 - mae: 50270.8906
Test MAE : 50270.890625


In [25]:
model.save('regression_model.h5')

c:\Users\shukl\anaconda3\envs\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
